In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("neuronetties/durak")

# Формат датасета

Датасет представляет из себя массив строк последовательных снапшотов. У каждой записи есть `game_id` - идентификатор игры и `snapshot` - снимок состояния игры представленный в виде JSON. У каждой игры есть несколько снапшотов (если `game_id` одинаковый, значит снапшоты принадлежат одной игре). Каждый снапшот имеет в себе значение `timestamp` - порядковый номер снапшота внутри.

In [ ]:
# Выведем все снашоты относящиеся к одной из игр по ID.

game_id_to_find = "8219abbc-7ec9-48f5-b9be-63374c227d6d"
filtered_dataset = dataset.filter(lambda example: example['game_id'] == game_id_to_find)

for snapshot in filtered_dataset['train']:
  print(snapshot)

{'game_id': '8219abbc-7ec9-48f5-b9be-63374c227d6d', 'snapshot': '{"game_id": "8219abbc-7ec9-48f5-b9be-63374c227d6d", "trump": "14S", "timestamp": 0, "winner": "7399412313", "game_rules": {"game_type": 1, "player_amount": 2, "card_amount": 24}, "deck": ["14S", "13D", "9C", "14H", "11S", "11C", "10C", "12H", "9D", "11H", "12S", "9H"], "bat": [], "table": [], "players": [{"id": "7399412313", "state": "attack", "hand": ["10D", "12D", "13C", "14C", "9S", "13S"]}, {"id": "224737994", "state": "defend", "hand": ["11D", "14D", "12C", "10H", "13H", "10S"]}]}'}
{'game_id': '8219abbc-7ec9-48f5-b9be-63374c227d6d', 'snapshot': '{"game_id": "8219abbc-7ec9-48f5-b9be-63374c227d6d", "trump": "14S", "timestamp": 1, "winner": "7399412313", "game_rules": {"game_type": 1, "player_amount": 2, "card_amount": 24}, "deck": ["14S", "13D", "9C", "14H", "11S", "11C", "10C", "12H", "9D", "11H", "12S", "9H"], "bat": [], "table": [{"attack_card": {"card": "10D", "user_id": "7399412313"}}], "players": [{"id": "739941

In [ ]:
# Переведем JSON снапшоты в человеческий язык.

import json

# Получаем первый снапшот
first_snapshot_data = json.loads(filtered_dataset['train'][0]['snapshot'])

# Общая информация об игре (смотрим по первому снапшоту, т.е. общая информация во всех снапшотах с одинаковым game_id - одинаковая)
print("=== Общая информация об игре ===")
print(f"ID игры: {first_snapshot_data['game_id']}")
print(f"Режим: {'Классический' if first_snapshot_data['game_rules']['game_type'] == 0 else 'Переводной'}")
print(f"Козырь: {first_snapshot_data['trump']}")
print(f"Количество игроков: {first_snapshot_data['game_rules']['player_amount']}")
print(f"Количество карт в колоде: {first_snapshot_data['game_rules']['card_amount']}")
print(f"Победитель: {first_snapshot_data['winner']}")
print()

# Состояния по каждому снапшоту
print("=== Ход игры по снапшотам ===")
for snap in filtered_dataset['train']:
    data = json.loads(snap['snapshot'])
    print(f"\n--- Снапшот t={data['timestamp']} ---")

    # Игроки и их карты
    for player in data['players']:
        role = {
            "attack": "Атакует",
            "defend": "Обороняется",
            "bat": "Ход окончен (бито)",
            "take": "Берёт карты",
            "pass": "Пропускает",
            "winner": "Победил",
            "durak": "Проиграл",
        }.get(player["state"], player["state"])
        print(f"Игрок {player['id']} — {role}")
        print(f"  Карты на руке: {', '.join(player['hand']) if player['hand'] else 'Нет карт'}")

    # Карты на столе
    if data["table"]:
        print("На столе:")
        for pair in data["table"]:
            attack = pair['attack_card']['card']
            attacker = pair['attack_card']['user_id']
            defend = pair.get('defend_card')
            if defend:
                defend_card = defend['card']
                defender = defend['user_id']
                print(f"  {attacker} атакует {attack}, {defender} отбивается {defend_card}")
            else:
                print(f"  {attacker} атакует {attack}, защита ещё не сыграна")
    else:
        print("На столе: пусто")

    # Колода
    print(f"Колода: {len(data['deck'])} карт" + (f" ({', '.join(data['deck'])})" if data['deck'] else ""))

    # Бито
    print(f"Бито: {len(data['bat'])} карт" + (f" ({', '.join(data['bat'])})" if data['bat'] else ""))


=== Общая информация об игре ===
ID игры: 8219abbc-7ec9-48f5-b9be-63374c227d6d
Режим: Переводной
Козырь: 14S
Количество игроков: 2
Количество карт в колоде: 24
Победитель: 7399412313

=== Ход игры по снапшотам ===

--- Снапшот t=0 ---
Игрок 7399412313 — Атакует
  Карты на руке: 10D, 12D, 13C, 14C, 9S, 13S
Игрок 224737994 — Обороняется
  Карты на руке: 11D, 14D, 12C, 10H, 13H, 10S
На столе: пусто
Колода: 12 карт (14S, 13D, 9C, 14H, 11S, 11C, 10C, 12H, 9D, 11H, 12S, 9H)
Бито: 0 карт

--- Снапшот t=1 ---
Игрок 7399412313 — Атакует
  Карты на руке: 12D, 13C, 14C, 9S, 13S
Игрок 224737994 — Обороняется
  Карты на руке: 11D, 14D, 12C, 10H, 13H, 10S
На столе:
  7399412313 атакует 10D, защита ещё не сыграна
Колода: 12 карт (14S, 13D, 9C, 14H, 11S, 11C, 10C, 12H, 9D, 11H, 12S, 9H)
Бито: 0 карт

--- Снапшот t=2 ---
Игрок 7399412313 — Атакует
  Карты на руке: 12D, 13C, 14C, 9S, 13S
Игрок 224737994 — Обороняется
  Карты на руке: 14D, 12C, 10H, 13H, 10S
На столе:
  7399412313 атакует 10D, 224737994 

# Формат данных для отправки на сервер (доступные метода)

Структуры пакетов, отправляемых на сервер - вам не нужно это менять, просто показываем для понимания

In [ ]:
# 1. Смена состояния (State)
state_pass = {"type": "state", "state": "pass"}
state_bat = {"type": "state", "state": "bat"}
state_take = {"type": "state", "state": "take"}

# 2. Атака или перевод (Attack)
# Пример: игрок атакует/переводит картой "10H"
attack_move = {"type": "attack", "move": "10H"}

# 3. Защита (Defend)
# Пример: защищаем "10H" картой "11H", и "9S" картой "12S"
defend_move = {
    "type": "defend",
    "move": [
        ["123123", "10H"],  # где "123123" — ID игрока, которые атакует, и карта "10H", которой атакуют
        ["321321", "11H"]   # где "321321" - ID игрока, который защищается (т.е. мы), и "11H" — карта, которой отбиваемся
    ]
}

# 4. Ожидание (Wait)
wait_packet = {"type": "wait"}


Пример клиента и методов, которые будут в нём. Его тоже менять не нужно, полноценный клиент появится **12.04**, когда мы откроем вам доступ к нему. Здесь вам важны только методы, которые вы можете вызвать.

In [ ]:
class GameClient:
    def __init__(self):
        self.my_id = "example-id-123" # ID игрока (бота), за которого играем
        self.table = [
            {"attack_card": {"card": "10H", "user_id": "123"}},
            {"attack_card": {"card": "9S", "user_id": "123"}}
        ] # Стол. Он будет обновляться динамические сам, вам запариваться не надо

# --- --- --- Основные методы, которые может вызвать нейросеть, вам их менять не надо --- --- ---

    def send_pass(self):
        packet = {"type": "state", "state": "pass"}
        print(f"Отправлено действие: {packet}")

    def send_bat(self):
        packet = {"type": "state", "state": "bat"}
        print(f"Отправлено действие: {packet}")

    def send_take(self):
        packet = {"type": "state", "state": "take"}
        print(f"Отправлено действие: {packet}")

    def send_attack(self, card: str): # Атака и Перевод
        packet = {"type": "attack", "move": card}
        print(f"Отправлено действие: {packet}")

    def send_defend(self, attack_card: str, defend_card: str):
        attacker_id = None
        for pair in self.table:
            if pair.get("attack_card", {}).get("card") == attack_card:
                attacker_id = pair["attack_card"]["user_id"]
                break

        if not attacker_id:
            print(f"Ошибка: карта {attack_card} не найдена на столе.")
            return

        packet = {
            "type": "defend",
            "move": [
                [attacker_id, attack_card],
                [self.my_id, defend_card]
            ]
        }
        print(f"Отправлено действие: {packet}")

    def send_wait(self):
        packet = {"type": "wait"}
        print(f"Отправлено действие: {packet}")


In [ ]:
client = GameClient()

client.send_pass()
client.send_bat()
client.send_take()
client.send_attack("14D")
client.send_defend("10H", "11H")
client.send_wait()

Отправлено действие: {'type': 'state', 'state': 'pass'}
Отправлено действие: {'type': 'state', 'state': 'bat'}
Отправлено действие: {'type': 'state', 'state': 'take'}
Отправлено действие: {'type': 'attack', 'move': '14D'}
Отправлено действие: {'type': 'defend', 'move': [['123', '10H'], ['example-id-123', '11H']]}
Отправлено действие: {'type': 'wait'}
